In [ ]:
from pystac_client import Client
from odc.stac import configure_rio, stac_load

Configure the bands

In [2]:
config = {
    "s2_l2a": {
        "assets": {
            "*": {
                "data_type": "uint16",
                "nodata": 0,
                "unit": "1",
            },
            "SCL": {
                "data_type": "uint8",
                "nodata": 0,
                "unit": "1",
            },
        },
        "aliases": {
            "costal_aerosol": "B01",
            "blue": "B02",
            "green": "B03",
            "red": "B04",
            "red_edge_1": "B05",
            "red_edge_2": "B06",
            "red_edge_3": "B07",
            "nir": "B08",
            "nir_narrow": "B08A",
            "water_vapour": "B09",
            "swir_1": "B11",
            "swir_2": "B12",
            "mask": "SCL",
            "aerosol_optical_thickness": "AOT",
            "scene_average_water_vapour": "WVP",
        },
    }
}

In [3]:
configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
    AWS_S3_ENDPOINT="s3.af-south-1.amazonaws.com",
)

In [4]:
# Open the stac catalogue
catalog = Client.open("https://explorer.digitalearth.africa/stac")

In [6]:
# Set a bounding box
# [xmin, ymin, xmax, ymax] in latitude and longitude
bbox = [13.10, 13.0, 13.99, 14.0]

# Set a start and end date
start_date = "2024-05-01"
end_date = "2025-07-01"

# Set the STAC collections
collections = ["s2_l2a"]

In [7]:
# Build a query with the set parameters
query = catalog.search(
    bbox=bbox, collections=collections, datetime=f"{start_date}/{end_date}"
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")

Found: 576 datasets


In [8]:
crs = "EPSG:6933"
resolution = 20

ds = stac_load(
    items,
    bands=("SCL",),
    crs=crs,
    resolution=resolution,
    chunks={},
    groupby="solar_day",
    stac_cfg=config,
    bbox=bbox,
)

# View the Xarray Dataset
ds

<xarray.Dataset> Size: 5GB
Dimensions:      (y: 6208, x: 4295, time: 179)
Coordinates:
  * y            (y) float64 50kB 1.769e+06 1.769e+06 ... 1.645e+06 1.645e+06
  * x            (x) float64 34kB 1.264e+06 1.264e+06 ... 1.35e+06 1.35e+06
    spatial_ref  int32 4B 6933
  * time         (time) datetime64[ns] 1kB 2024-05-02T09:36:57 ... 2025-07-01...
Data variables:
    SCL          (time, y, x) uint8 5GB dask.array<chunksize=(1, 6208, 4295), meta=np.ndarray>

In [ ]:
ds["water"] = (ds.SCL == 6)


ds.water.compute().plot(col="time", col_wrap=6, vmin=0, vmax=1)

Matplotlib is building the font cache; this may take a moment.


: 